In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install xlrd

In [ ]:
pip install mesa

In [ ]:
from mesa import Agent
from mesa import Model
from mesa.time import BaseScheduler
from mesa.datacollection import DataCollector
import random as rd
import numpy as np
import pandas as pd
import numpy as np
import random as rd
import numpy as np


cost_depot_customers = pd.read_excel(r"/kaggle/input/data-sma/6_detail_table_cust_depots_distances.xls")
cost_customers_customers = pd.read_excel(r"/kaggle/input/data-sma/7_detail_table_cust_cust_distances.xls")
trucks_infos = pd.read_excel(r"/kaggle/input/data-sma/3_detail_table_vehicles.xls")
depot_infos= pd.read_excel(r"/kaggle/input/data-sma/4_detail_table_depots.xls")
customers_list = pd.read_excel(r"/kaggle/input/data-sma/2_detail_table_customers.xls")
road_choice = 2922001

###############################################################################
#################################   Classes   #################################
###############################################################################

class Client:

    def __init__(self, name, x, y, quantity, start, stop,code):
        self.name = name
        self.code = code
        self.x = x
        self.y = y
        self.quantity = quantity
        self.start = start
        self.stop = stop
        self.delivered = False

class Truck:
    
    def __init__(self, name, quantity_max, start, stop,route_id):
        self.name = name
        self.route_id = route_id
        self.x = 0
        self.y = 0
        self.quantity_max = quantity_max
        self.start = start
        self.stop = stop
        self.remaining_quantity=quantity_max
        self.P = [0]
        self.cost = 0
        self.time = 0

    def delivery(self, client):
        self.x=client.x
        self.y=client.y
        self.remaining_quantity -= client.quantity
        
    def calculate_cost(self, data,listOfClients):
        self.cost=0
        self.time=0
        self.remaining_quantity=self.quantity_max
        demande = 0
        weight_K = 10000
        weight_q = 10000
        weight_t = 3
        weight_d = 13
        weight_c = 10     
        for m in range(len(self.P)):
            demande += listOfClients[m].quantity
        self.remaining_quantity-=demande
        if(self.remaining_quantity<0):
            self.cost+=(self.remaining_quantity*(-1))*weight_q
        for i in range (len(self.P)-1):
            self.cost += (distanceTab(self.P[i],self.P[i+1],listOfClients)*weight_c)
            self.time += data[self.P[i]][self.P[i+1]]
            if self.time <= listOfClients[self.P[i+1]].start:
                self.cost += ((listOfClients[self.P[i+1]].start - self.time)*weight_t)
                self.time = listOfClients[self.P[i+1]].start
            if self.time >= listOfClients[self.P[i+1]].stop:
                self.cost += ((self.time - listOfClients[self.P[i+1]].stop)*weight_d)
        self.cost += weight_K
        return (self.cost)
        

           


def distanceTab(C1, C2, listOfClients):
        return(np.sqrt((listOfClients[C1].y-listOfClients[C2].y)**2 + (listOfClients[C1].x - listOfClients[C2].x)**2))
    


def find_client_by_code(code,liste_clients):
    find = False
    i=0
    client = Client(-1,-1,-1,-1,-1,-1,-1)
    while i<len(liste_clients) and not find :
        if liste_clients[i].code == code :
            client = liste_clients[i]
            find = True
        i +=1
    return client


def cost_matrix(liste_clients,cost_depot_customers,cost_customers_customers):
    n = len(liste_clients)+1
    cost_matrix= np.zeros((n,n))
    time_matrix= np.zeros((n,n))
    for i in range(0,len(cost_depot_customers),2):
        client = find_client_by_code(cost_depot_customers['CUSTOMER_CODE'][i],liste_clients)
        if client.name == -1 :
            pass
        else : 
            cost_matrix[0][client.name+1] = cost_depot_customers['DISTANCE_KM'][i]
            cost_matrix[client.name+1][0] = cost_depot_customers['DISTANCE_KM'][i+1]
            time_matrix[0][client.name+1] = cost_depot_customers['TIME_DISTANCE_MIN'][i]
            time_matrix[client.name+1][0] = cost_depot_customers['TIME_DISTANCE_MIN'][i+1]
    for i in range(len(cost_customers_customers)):
        client_1=find_client_by_code(cost_customers_customers['CUSTOMER_CODE_FROM'][i],liste_clients)
        client_2=find_client_by_code(cost_customers_customers['CUSTOMER_CODE_TO'][i],liste_clients)
        if client_1.name == -1 or client_2.name==-1 : 
            pass
        else : 
            cost_matrix[client_1.name+1][client_2.name+1] = cost_customers_customers['DISTANCE_KM'][i]
            time_matrix[client_1.name+1][client_2.name+1] = cost_customers_customers['TIME_DISTANCE_MIN'][i]
    return cost_matrix,time_matrix

def liste_c(customers_list):
    liste_clients = []
    for i in range(len(customers_list)):
        customer = Client(i,customers_list['CUSTOMER_LATITUDE'][i],customers_list['CUSTOMER_LONGITUDE'][i],
                        customers_list['CUSTOMER_TIME_WINDOW_FROM_MIN'][i],customers_list['CUSTOMER_TIME_WINDOW_TO_MIN'][i],customers_list['TOTAL_VOLUME_M3'][i],
                        customers_list['CUSTOMER_CODE'][i])
        liste_clients.append(customer)
    return liste_clients

def liste_tr(trucks_infos,depot_infos):
    lat = depot_infos['DEPOT_LATITUDE'][0]
    long = depot_infos['DEPOT_LONGITUDE'][0]
    T = []
    for i in range(len(trucks_infos)):
        T.append(Truck(i,trucks_infos['VEHICLE_TOTAL_VOLUME_M3'][i],lat,long,trucks_infos['ROUTE_ID'][i]))
    return T


def truncate_df(df,road_choice):
    minimum = df.ROUTE_ID.eq(road_choice).idxmax()
    df = df.truncate(before = minimum)
    maximum = df.ROUTE_ID.ne(road_choice).idxmax()-1
    df = df.truncate(after=maximum)
    df.reset_index(drop=True,inplace=True)
    return df

cost_customers_customers=truncate_df(cost_customers_customers,road_choice)
cost_depot_customers=truncate_df(cost_depot_customers,road_choice)
trucks_infos=truncate_df(trucks_infos,road_choice)
customers_list=truncate_df(customers_list,road_choice)
depot_infos = truncate_df(depot_infos,road_choice)

depot=Client(-1,depot_infos['DEPOT_LATITUDE'][0],depot_infos['DEPOT_LONGITUDE'][0],0,0,0,-100000)

list_clients = liste_c(customers_list)
clients= [[list_clients[i].quantity, list_clients[i].start, list_clients[i].stop] for i in range(len(list_clients))]
trucks_list = liste_tr(trucks_infos,depot_infos)
n_trucks = len(trucks_list)
(c_matrix,t_matrix)=cost_matrix(list_clients,cost_depot_customers,cost_customers_customers)


###############################################################################
######################   Enregistrement des constantes   ######################
###############################################################################
T = liste_tr(trucks_infos,depot_infos)
n_trucks = len(T) #Nombre de camions
truck_capacity = [ i.quantity_max for i in T] #Capacité unitaire pouvant être emmenée par chaque camion
time_matrix = t_matrix

#Paramètres AGs
nb_pop = 100 #taille de notre population
nb_generations = 200 #nombre de générations étudiées par l'AGs
elitism = True #Volonté de ne sélectionner que les meilleurs éléments
best_pop = 40 #Taille de la population élite : les N-meilleurs membres de la population
mutation_rate = 0.4 #probabilité de mutation lors du passage à la génération suivante

weight_K=10_000
weight_q=10_000
weight_t=3
weight_d=13
weight_c=10

###############################################################################
##########################   Fonctions d'évaluation  ##########################
###############################################################################



def track_evaluation(truck_track):
    truck_track = closing_tour(truck_track)
    t3=0
    trucks_time = []
    for track in truck_track:
        t=0
        for q in truck_capacity :
            
            if len(track) >2 :
                t3+= weight_K*len(truck_capacity) #Si le camion est utilisé, ajout du coût d'utilisation dans le score du circuit
            for i in range(1,len(track)):
                
                # Evaluation fenêtre
                t2= time_matrix[track[i-1]][track[i]]
                
                t4 = c_matrix[track[i-1]][track[i]]
                if t+t2 <= clients[track[i]][1]: #Si arrivé du livreur avant la fin de la fenêtre
                    t+=t4 * weight_c
                    t3 +=  max(0,clients[track[i]][0]-t) * weight_t
                    t+=+max(0,clients[track[i]][0]-t)  #ajout d'un temps d'attente si le livreur arrive avant le début de la fenêtre
                else :
                    t3+= (t+t2 - clients[track[i]][1])*weight_d #ajout d'une pénalité si une fenêtre n'est pas respectée
                    t+=t4*weight_c
                  
                # Evaluation quantité            
                if q >= clients[track[i]][2]:
                    q-=clients[track[i]][2]
                else : 
                    t3+=(clients[track[i]][2]-q)*weight_q   #ajout d'une pénalité si une quantité n'est pas respectée
                    q=0                
        trucks_time.append(t)
        t3+=t
    return t3 ,trucks_time
    
                
def truck_track_constructor(member):
    track=member[0]   


    cgt=[0]
    truck_track=[]
    
    for j in range(1,len(track)):
        if track[j] <= 0:           
            cgt.append(j)
            truck_track.append([0] + track[cgt[-2]+1:cgt[-1]] + [0])
            
    cgt.append(j)
    truck_track.append([0] + track[cgt[-2]+1:cgt[-1]+1] + [0])  

    return truck_track

def population_evaluation(member): 
    t3,trucks_time=track_evaluation(truck_track_constructor(member))        
    member[1]=t3
    return member

#Nous avons reprogrammé ci-dessous un tri fusion
def merge(left,right): 
    result = []
    index_left, index_right = 0, 0
    while index_left < len(left) and index_right < len(right):        
        if left[index_left][1] <= right[index_right][1]:
            result.append(left[index_left])
            index_left += 1
        else:
            result.append(right[index_right])
            index_right += 1
    if left:
        result.extend(left[index_left:])
    if right:
        result.extend(right[index_right:])
    return result
 
def merge_sort(m):
    if len(m) <= 1:
        return m
    middle = len(m) // 2
    left = m[:middle]
    right = m[middle:]
    left = merge_sort(left)
    right = merge_sort(right)
    return list(merge(left, right))

def track_to_member(track):
    i=0
    member=[[],track_evaluation(track)[0]]
    for truck_track in track:
        member[0]= member[0]+[i]+truck_track[1:-1]
        i-=1
    return(member)
    
###############################################################################
##########################   fonctions d'évolution   ##########################
###############################################################################

rand=rd.random

Phenon=[i for i in range(len(clients))] +[-i for i in range(1,n_trucks)]
back_to_depot=[-i for i in range(n_trucks)]

nv=len(Phenon)

def generate():
    a=Phenon[1:]
    rd.shuffle(a)
    return [0]+a

#def taille_pop2():
#    pop = []
#    for i in range(taille_pop):
#        circuit2=generate()
#        distance=0
#        for j in range(1,nv):
#            distance+=dist[circuit2[j-1],circuit2[j]]
#        pop.append([circuit2,distance])
#    return pop
        
    

def crossover(parent1,parent2):

    parent1 = parent1[1:]
    parent2 = parent2[1:]
    nv=len(parent1)
    # On récupère une partie des critères du premier parent
    start_point=rd.randint(0,nv-1)
    end_point=rd.randint(0,nv-1)
    
    if start_point > end_point :
        start_point,end_point=end_point,start_point
    
    child=["False" for i in range(nv)]
    heritage_parent1=parent1[start_point:end_point+1]
    child[start_point:end_point+1]=heritage_parent1
    
    
    # On récupère ensuite le maximum de critères possibles du parent2 tout en conservant un enfant sans doublons
    m1=[]
    for i in range(nv):
        if i > end_point or i < start_point: 
            criteria=parent2[i]
            if criteria not in heritage_parent1:
                child[i]=criteria
            else:
                m1.append(i)
                
                
    # On complète par les critères manquants            
    m2=[]
#    print(enfant)
    for i in Phenon[1:]:

        
        if i not in child:
            m2.append(i)

    rd.shuffle(m2)
#    print(patrimoine_parent1)
#    print(parent2)
#    print(m2)
#    print(m1)
#    print(enfant)
    for i in range(len(m1)):
        child[m1[i]]=m2[i]
    
    #mutation

    
    if rd.random() < mutation_rate:
        a=rd.randint(0,len(child)-1)
        b=rd.randint(0,len(child)-1)   
        child[a], child[b] = child[b], child[a]
            
    return [0]+child 

def uniform_cross(p1,p2):
    
    p1 = p1[1:]
    p2 = p2[1:]
    
    child1 = []
    child2 = []
    if len(p1) != len(p2):
        return(generate(),generate())

    for i in range(len(p1)):
        if(p1[i] in child1 and p2[i] in child1):
            child2.append(p1[i])
            child2.append(p2[i])
        elif(p1[i] in child2 and p2[i] in child2):
            child1.append(p1[i])
            child1.append(p2[i])
        elif(p1[i] in child2 or p2[i] in child1):
            child1.append(p1[i])
            child2.append(p2[i])

        elif(p1[i] in child1 or p2[i] in child2):
            child1.append(p2[i])
            child2.append(p1[i])

        else:
            a = rd.randint(1,2)
            if(a == 1):
                child1.append(p1[i])
                child2.append(p2[i])
            else:
                child1.append(p2[i])
                child2.append(p1[i])
    child1=mutation(child1, mutation_rate)
    child2=mutation(child2, mutation_rate)

    return([0]+child1, [0] + child2)

def mutation(child, mutation_rate):
    if(rd.random()-mutation_rate < 0):
        point1 = rd.randint(0, len(child)-1)
        point2 = rd.randint(0, len(child)-1)
        child[point1], child[point2] = child[point2], child[point1]
    return(child)
        
def init_pop(n):
    pop=[]
    for i in range(n):
        track=generate()
        pop.append(population_evaluation([track,0]))
    pop=merge_sort(pop)
    return pop



def next_gen(population):
  
    new_gen=[]
    
    if elitism == True:
        elite=population[:best_pop]
    else:
        elite=[]
        

    nb_child=(nb_pop//2)
    
    couples=[i for i in range(nb_child*2)]

    rd.shuffle(couples)


    for i in range(nb_child//2):
        
        # child=crossover(population[couples[i]][0],population[couples[nb_pop//2+i]][0])        
        # new_gen.append(population_evaluation([child,0]))
        
        child1=uniform_cross(population[couples[i]][0],population[couples[nb_pop//2+i]][0])[0]
        child2=uniform_cross(population[couples[i]][0],population[couples[nb_pop//2+i]][0])[1]
        new_gen.append(population_evaluation([child1,0]))
        new_gen.append(population_evaluation([child2,0]))
        
    new_gen=new_gen+elite
    
    new_gen= new_gen + init_pop(nb_pop-len(new_gen))
    
    population=merge_sort(new_gen[:])
    
    return(population)

    
################################## Utility functions ##################################

def find_client_by_name(name,C):
    i = 0
    n = len(C)
    find = False
    client = Client(-1,0,0,0,0,0,-1)
    while i < n and find == False : 
        if C[i].name == name :
            client = C[i]
            find = True
        i +=1
    return client


def quantity(C):
    q = 0
    for i in range(len(C)):
        q += C[i].quantity
    return q

def copy(F):
    L = []
    for x in F:
        V = []
        for i in range(len(x)):
            V.append(x[i])
        L.append(V)
    return L

def closing_tour(itineraire):
    test = copy(itineraire)
    for x in test:
        x.insert(0,0)
        if x[-1]!=0:
            x.append(0)
        del(x[1])
    return test

def convert_solution(solution, trucks_list):
    if solution[0][0]==0:
        for i in range(len(solution)):
            solution[i][0]=trucks_list[i]
            del(solution[i][-1])
    return solution
closing_tour
#################################### RS Algorithm #####################################
def random_solution(C, trucks_list):
    C2 = [client.name for client in C]
    q = quantity(C)
    T = trucks_list
    t = len(T)
    R = []
    C_bis = C2.copy()
    for i in range(t-1):
        s = rd.sample(C_bis,rd.randint(1,len(C_bis)-(t-i+1)))
        R.append([T[i]]+s)
        for x in s :
            C_bis.remove(x)
    rd.shuffle(C_bis)
    L  = [T[t-1]]+C_bis
    R.append(L)
    for i in range(t,len(T)):
        R.append([T[i]])
    return R

def neighbouring_solution(R):
     n = len(R)
     S = copy(R)
     i = rd.randint(0,n-1)
     j = rd.randint(0,n-1)
     l = rd.randint(1,len(S[i]))
     k = rd.randint(1,len(S[j]))
     if l == len(S[i]) and k == len(S[j]):
         return S
     elif k == len(S[j]) :
         client = S[i][l]
         S[j].append(client)
         S[i].remove(client)
     elif l == len(S[i]) : 
        client = S[j][k]
        S[i].append(client)
        S[j].remove(client)
     else:         
         client1 = S[i][l]
         client2 = S[j][k]
         S[i][l] = client2
         S[j][k] = client1
     return S

def cost_function(R,M,M_time,C, weight_K, weight_q, weight_t, weight_d, weight_c):
    T = 0
    n = len(R)
    K = 0
    for x in R :
        if len(x)> 1 :
            K+=1
    t = 0
    d = 0
    c = 0
    q = 0
    dispo = [R[i][0].start for i in range(n) ]
    for i in range(n):
        if len(R[i])> 1 :
            truck = R[i][0]
            truck.remaining_quantity = truck.quantity_max
            client = find_client_by_name(R[i][1],C)
            dispo[i] = dispo[i] + M_time[client.name][0]
            c+= M[client.name][0]
            d += max(0,dispo[i] - client.stop)
            t -= min(0, dispo[i] - client.start)
            truck.delivery(client)
            for j in range(2,len(R[i])):
                client_prec = find_client_by_name(R[i][j-1],C)
                client = find_client_by_name(R[i][j],C)
                c+= M[client_prec.name][client.name]
                dispo[i] = dispo[i] + M_time[client_prec.name][client.name]
                d += max(0,dispo[i] - client.stop)
                t -= min(0, dispo[i] - client.start)
                truck.delivery(client)
            c += M[0][client.name]
            q -= min(0,truck.remaining_quantity)
    T = weight_K * K + weight_q * q + weight_t * t + weight_d * d + weight_c * c
    return T

def algo_RS(C,M,M_time, trucks_list, start_solution, n=10000, weight_K=1_000, weight_q=1_000, weight_t=30, weight_d=500, weight_c=0.1): #n=10_000
    R = convert_solution(start_solution, trucks_list)
    T = cost_function(R,M,M_time,C, weight_K, weight_q, weight_t, weight_d, weight_c)
    i = 0
    while  i < n :
        S = neighbouring_solution(R)
        T_bis = cost_function(S,M,M_time,C, weight_K, weight_q, weight_t, weight_d, weight_c)
        p = (T_bis - T)*100000
        random = rd.random()
        if p<0 or random < np.exp(-p/T):
            R = copy(S)
            T = T_bis
        i+= 1
    R=closing_tour(R)
    return (R,T_bis)

###################################### Indicators #######################################
def delay_indicator(R,C,M_time):
    dispo = [0]* len(R)
    delay_by_truck = [0]* len(R)
    clients_name = []
    cumulative_delay = 0
    for x in R:
        if len(x)>1:
            c= find_client_by_name(x[1],C)
            dispo[x[0].name] += M_time[c.name][0]
            if dispo[x[0].name]-c.stop > 0 :
                delay_by_truck[x[0].name] += dispo[x[0].name]-c.stop
                clients_name.append(c.name)
                cumulative_delay += dispo[x[0].name]-c.stop
            for i in range(2,len(x)) :
                c= find_client_by_name(x[i],C)
                dispo[x[0].name] += M_time[c.name][x[i-1]]
                if dispo[x[0].name]-c.stop > 0 :
                    clients_name.append(c.name)
                    delay_by_truck[x[0].name] += dispo[x[0].name]-c.stop
                    cumulative_delay +=dispo[x[0].name]-c.stop
    return delay_by_truck, clients_name, cumulative_delay


def advance_indicator(R,C,M_time):
    dispo = [0]* len(R)
    advance_by_truck = [0] * len(R)
    cumulative_advance = 0
    for x in R:
        if len(x)>1:
            c= find_client_by_name(x[1],C)
            dispo[x[0].name] += M_time[c.name][0]
            if c.start - dispo[x[0].name] > 0 :
                advance_by_truck[x[0].name] += c.start - dispo[x[0].name]
                cumulative_advance += c.start - dispo[x[0].name]
            for i in range(2,len(x)) :
                c= find_client_by_name(x[i],C)
                dispo[x[0].name] += M_time[c.name][x[i-1]]
                if dispo[x[0].name]-c.stop > 0 :
                    advance_by_truck[x[0].name] +=c.start - dispo[x[0].name]
                    cumulative_advance +=c.start - dispo[x[0].name]
    return advance_by_truck, cumulative_advance

###############################################################################
###################################   SMA   ###################################
###############################################################################
class gen_agent(Agent):
    
    def __init__(self,unique_id,model, nb_pop, nb_generations, n_truck, truck_capacity, list_clients):
        super().__init__(unique_id,model)
        self.nb_pop = nb_pop
        self.nb_generations =  nb_generations
        self.n_trucks = n_trucks
        self.truck_capacity = truck_capacity
        self.list_clients = list_clients
        self.time_matrix = t_matrix     
        self.best_solution = [] # Meilleure solution à retourner en fin d'éxécution        
        self.population = init_pop(nb_pop)
        self.solution = truck_track_constructor(self.population[0])
        
        
    
def step(self) :
    pool_rate = 0.5 # Probabilité d'utiliser le Taboo
    pool_rate2 = 0.7 # Probabilité d'intégrer le n ième ordonnancement du pool (s'il est utilisé) à la population initiale

    # Régénération de la population
    self.population = init_pop(self.nb_pop)

    if len(self.model.pool) > 0 and rd.random() < pool_rate:
        L = []
        for track in self.model.pool :
            if rd.random() < pool_rate2 :
                L.append(track_to_member(closing_tour(track)))
        n_L = len(L)

        # Intégration des éléments du pool à la nouvelle population
        for i in range(n_L):
            self.population[i] = L[i]   

    for i in range(2,nb_generations+1):
        self.population = next_gen(self.population)

    # Mise en forme de l'ordonnancement 
    self.solution = truck_track_constructor(self.population[0])
        
class tab_agent(Agent):
    class Depot:
        def __init__(self, num_truck,trucks_list):
            self.x = 0
            self.y = 0
            self.num_truck = num_truck
            self.T = trucks_list
            

    def __init__(self, name, model):
        super().__init__(name,model)
        self.solution = [] # Meilleure solution à retourner en fin d'éxécution
        


    def solution_initiale (self,num_truck,listOfClients,data):
        for a in self.model.schedule.agents:
            if len(self.model.pool)>0:
                i = rd.randint(0, len(self.model.pool)-1)
                # print(i)
                # print(self.model.pool[i].copy())
                return track_evaluation(closing_tour(self.model.pool[i].copy()))[0],closing_tour(self.model.pool[i].copy())
            if len(self.model.pool) ==self.model.n_pool:
                i = rd.randint(0, self.model.n_pool-1)
                return track_evaluation(closing_tour(self.model.pool[i].copy()))[0],closing_tour(self.model.pool[i].copy())
        S=[]
        depot = self.Depot(num_truck,self.model.trucks_list)
        clients = [i for i in range (1,len(listOfClients))]
        while len (clients) != 0:
            for k in range (len (depot.T)):
                if len (clients) != 0:
                    j = rd.randint (0,len(clients)-1)
                    depot.T[k].P.append(clients[j])
                    clients.remove(clients[j])

        for k in range (len(depot.T)):
            depot.T[k].P.append(0)
        for i in range (num_truck):
            S.append(depot.T[i].P)
        #print(self.total_cost(depot,data,listOfClients),S)
        #return (self.total_cost(depot,data,listOfClients),S)
        return track_evaluation(S)[0],S
    
                    
    def total_cost (self,depot, data,listOfClients):
        total_cost = 0
        for i in range (depot.num_truck):
            (depot.T[i]).calculate_cost(data,listOfClients)
            total_cost += depot.T[i].cost
        return total_cost

    def simple_permut (self,P, old, new):
        temp = P[old]
        P.remove(P[old])
        P.insert(new,temp)
        return P

    def voisinage_simple (self,parcours, truck, data,listOfClients):
        longueurVoisinage = 0
        longueurMax = 300
        copie_parcours=parcours+[]
        vs=[parcours]
        truck.P = parcours
        cost=[truck.calculate_cost(data,listOfClients)]

        for i in range (1,len(copie_parcours)-1):
            for j in range (1,len(copie_parcours)-1):
                if i < j and longueurVoisinage < longueurMax:
                    copie_parcours=self.simple_permut(copie_parcours,i,j)
                    vs.append(copie_parcours)
                    #print("simple")
                    longueurVoisinage+=1
                    truck.P = copie_parcours
                    cost.append(truck.calculate_cost(data,listOfClients))
                    copie_parcours=parcours+[]
        mini=cost[0]
        index_opti=0
        #print(cost)
        for k in range (1,len(cost)):
            #print(cost[k])
            if cost[k]<mini:
                mini=cost[k]
                index_opti=k
        return vs[index_opti]

    
    def voisinage_complexe (self,ens_parcours, num_truck, data,tabou,listOfClients):
        longueurVoisinage = 0
        longueurMax = 300
        copie_ens_parcours=ens_parcours+[]
        vc=[]
        list_tot_cost=[]
        for i in range (len(ens_parcours)):
            if longueurVoisinage< longueurMax:
                for j in range (len(ens_parcours)):
                    if i != j:
                        for k in range (1,len(copie_ens_parcours[i])-1):
                            for l in range (1,len(copie_ens_parcours[j])-1):
                                (a,b) =self.transfert(copie_ens_parcours[i],copie_ens_parcours[j],k,l)
                                ens_parcours_to_add=[]
                                depot0 = self.Depot(num_truck,self.model.trucks_list)
                                for w in range (len(copie_ens_parcours)):
                                    if w == i :
                                        ens_parcours_to_add.append(a)
                                        depot0.T[w].P=a
                                    elif w == j :
                                        ens_parcours_to_add.append(b)
                                        depot0.T[w].P=b
                                    else:
                                        ens_parcours_to_add.append(copie_ens_parcours[w])
                                        depot0.T[w].P=copie_ens_parcours[w]
                                vc.append(ens_parcours_to_add)
                                list_tot_cost.append(track_evaluation(ens_parcours_to_add)[0])
                                longueurVoisinage+=1
                                #print("complexe")
        #print(list_tot_cost)
        return (list_tot_cost, vc)                                                  
                    
            
    def best_voisinage (self,num_truck, data,sol_actuelle,tabou,best_saved_cost,listOfClients):
        all_ens_vs=[]
        all_cost=[]
        for i in range (len(sol_actuelle)):# nombre de camions
            ens_vs=[]
            cost=0
            for j in range (len(sol_actuelle)):
                if i==j:
                    truck = Truck(num_truck+1, self.model.trucks_list[j].quantity_max,self.model.trucks_list[j].start,self.model.trucks_list[j].stop,self.model.trucks_list[j].route_id)
                    truck.P = self.voisinage_simple(sol_actuelle[i], truck, data,listOfClients)+[]
                    ens_vs.append(truck.P)
                    truck.cost = truck.calculate_cost(data,listOfClients)
                    # print(truck.cost)
                    cost+= truck.cost
                else:
                    truck = Truck(num_truck+1, self.model.trucks_list[j].quantity_max,self.model.trucks_list[j].start,self.model.trucks_list[j].stop,self.model.trucks_list[j].route_id)
                    truck.P = sol_actuelle[j]+[]
                    cost+= truck.calculate_cost(data,listOfClients)
                    ens_vs.append(sol_actuelle[j])
            
            all_ens_vs.append(ens_vs)
            all_cost.append(cost)

        (cost_vc,vc)=self.voisinage_complexe(sol_actuelle,num_truck,data,tabou,listOfClients)
        all_ens_vs = all_ens_vs+vc+[]
        all_cost = all_cost+cost_vc+[]
        #print(all_cost)
        (all_ens_vs,all_cost) = self.sol_filter(data,sol_actuelle,tabou,all_ens_vs,all_cost,best_saved_cost)
        if(len(all_ens_vs)>0):
            min_cost=all_cost[0]
            index_best_cost=0
            for i in range (len(all_cost)):
                if (all_cost[i] < min_cost):
                    min_cost = all_cost[i]
                    index_best_cost = i
            return (all_cost[index_best_cost],all_ens_vs[index_best_cost])
        else:
            return (0,[])
    
    def sol_filter(self,data,sol_actuelle,tabou,all_ens_vs,all_cost,best_saved_cost):
        new_sol =[]
        new_sol_cost=[]
        for i in range (len(all_ens_vs)):
            if((sol_actuelle,all_ens_vs[i])not in tabou or all_cost[i]<best_saved_cost):
                new_sol.append(all_ens_vs[i])
                new_sol_cost.append(all_cost[i])
        return (new_sol,new_sol_cost)

    def transfert (self,P1, P2, old_pos, new_pos):
        copy_P1=P1+[]
        copy_P2=P2+[]
        temp = copy_P1[old_pos]
        copy_P1.remove(copy_P1[old_pos])
        copy_P2.insert(new_pos,temp)
        return (copy_P1,copy_P2)  

    def algo_tabou (self,nb_iter, max_tabou_size,number_trucks,listOfClients):
        curr_cost =0
        curr_sol=[]
        tabou =[]
        data = self.model.time_matrix
        (curr_cost,curr_sol) = self.solution_initiale(number_trucks,listOfClients,data)
        #print("test")
        #print(curr_sol,track_evaluation(curr_sol)[0])
        best_sol=curr_sol+[]
        best_cost=curr_cost
        # print(curr_sol)
        # print(curr_cost)
        for i in range(nb_iter):
            (curr_cost,best_curr_neigh) = self.best_voisinage(number_trucks,data,curr_sol,tabou,best_cost,listOfClients)
            #print("curr_cost")
            curr_cost=track_evaluation(best_curr_neigh)[0]
            #print(curr_cost)
            if(curr_cost==0):
                break
            if(len(tabou)>=max_tabou_size):
                tabou.pop(0)
            tabou.append((best_curr_neigh,curr_sol))
            curr_sol=best_curr_neigh+[]
            if(curr_cost<best_cost):
                best_cost=curr_cost
                best_sol=curr_sol+[]
        #print("/////////////////////////////////////////////resultat_tabou ///////////////////////////////////////////////////////////////////////////////////")  
        #print(best_sol,best_cost,track_evaluation(best_sol)[0],len(self.model.pool))
        #print("/////////////////////////////////////////////end_tabou ///////////////////////////////////////////////////////////////////////////////////")
        #return best_sol,best_cost
        return best_sol,track_evaluation(best_sol)[0]
    
    def step(self):
        sol =(self.algo_tabou (20, 20,len(self.model.trucks_list),self.model.list_clients))
        self.solution = sol[0]
        
    
class rs_agent(Agent):

    def __init__(self, unique_id, model, trucks_list, list_clients):
        super().__init__(unique_id,model)
        self.trucks_list = trucks_list
        self.list_clients = list_clients
        self.time_matrix = model.time_matrix 
        self.solution = [] # Meilleure solution à retourner en fin d'éxécution

    def step(self):
        #prendre un solution aléatoirement dans le pool
        for a in self.model.schedule.agents:
            if isinstance(a,pool_agent):
                a.closing()
                if len(a.pool) ==a.nb_solutions:
                    i = rd.randint(0, a.nb_solutions-1)
                    sol, cost = algo_RS(self.model.list_clients,self.model.cost_matrix, self.model.time_matrix, self.trucks_list, a.pool[i])
                    self.solution = closing_tour(sol)
                else:
                    sol, cost = algo_RS(self.model.list_clients,self.model.cost_matrix, self.model.time_matrix, self.trucks_list, closing_tour(random_solution(self.model.list_clients, self.trucks_list)))
                    self.solution = closing_tour(sol)
                a.closing()
        
        
class pool_agent(Agent):
    def __init__(self, unique_id, model, nb_solutions,pool_radius):
        super().__init__(unique_id,model)
        self.nb_solutions = nb_solutions
        self.pool = []
        self.pool_id = []
        self.pr = pool_radius
        
    def distance(self,solution1,solution2):
        s1 = []
        s2=[]
        r = 0
        for x in solution1:
            for i in range(1,len(x)):
                s1.append((x[i-1],x[i]))
        for x in solution2:
            for i in range(1,len(x)):
                s2.append((x[i-1],x[i]))
        for x in s1 :
            if not x in s2:
                r += 1
        return 1-r/self.pr if r<self.pr else 0
    
    
    def eval_function(self,pool2):

        g = 0
        for i in range(len(pool2)-1) :
            for y in pool2[i+1:]:
                g+= self.distance(y,pool2[i])
        return g
    def closing(self):
        for x in self.pool :
            x = closing_tour(x)
            
            
    def solution(self,model):
        self.closing()
        for agent in model.schedule.agents:
            if isinstance(agent,tab_agent) or isinstance(agent,rs_agent) or isinstance(agent,gen_agent):
                solution = closing_tour(agent.solution)
                res = -1
                self.closing()
                if len(self.pool)< self.nb_solutions :
                    phi = 0
                    if len(self.pool) > 0 :
                        for x in self.pool:
                            phi += self.distance(x,solution)
                        if phi == 0:                      
                            
                            self.pool.append(solution)
                            self.pool_id.append(agent.unique_id)
                    else :
                        self.pool.append(solution)
                        self.pool_id.append(agent.unique_id)
                    self.closing()
                else :
                    self.closing()
                    pool = self.pool
                    g = self.eval_function(pool)
                    eval_t = track_evaluation(self.pool[0])
                    for i in range(len(pool)):
                        self.closing()
                        pool_test = pool.copy()
                        old_sol = copy(closing_tour(pool_test[i]))
                        pool_test[i] =closing_tour(solution)
                        g_bis = self.eval_function(pool_test)
                        solution = closing_tour(solution)
                        if g_bis == 0 and track_evaluation(old_sol) > track_evaluation(solution):
                            if track_evaluation(old_sol) >= eval_t :
                                res = i
                                eval_t = track_evaluation(old_sol)
                        if res != -1 :
                            self.pool_id[res] = agent.unique_id
                            self.pool[res]=closing_tour(solution)
                            self.closing()   
                            
                        
        self.closing()
        self.model.pool = self.pool
        self.model.closing()
        self.model.pool_id = self.pool_id
        
                        
    def step(self):
        self.solution(model)
        self.closing()
        score =  [ [ track_evaluation(self.model.pool[i])[0] , self.model.pool_id[i]] for i in range(len(self.model.pool))]
        score = sorted(score)
        print(score)
        print('')
        print('////////////////////////////////////////////////////////////////////////////////')
        print('')
      



#la classe SMA
class SMA_collab(Model):

    def __init__(self, nb_pop, nb_generations, trucks_list, list_clients,cost_matrix, time_matrix, n_pool,radius_pool):
        self.nb_pop = nb_pop
        self.nb_generations = nb_generations
        self.trucks_list = trucks_list
        self.list_clients = list_clients
        self.clients=[[list_clients[i].quantity, list_clients[i].start, list_clients[i].stop] for i in range(len(list_clients))]
        self.cost_matrix=cost_matrix
        self.time_matrix = time_matrix
        self.n_pool = n_pool
        self.radius_pool = radius_pool
        self.pool=[]
        self.pool_id = []
        #self.grid = MultiGrid(width, height, True)
            
        self.datacollector = DataCollector(          
            agent_reporters={"State": "state"})

        
        #l'ordonnanceur du modele (instance de RandomActivation)
        #tester SimultaneousActivation (qui permet d'activer tous les agents en mêê temps)
        self.schedule = BaseScheduler(self)

        e = gen_agent(1,self,self.nb_pop, self.nb_generations, n_trucks, truck_capacity,self.list_clients)
        self.schedule.add(e)
        
        b = tab_agent(2,self)
        self.schedule.add(b)
        
        c = rs_agent(3, self, trucks_list, self.list_clients)
        self.schedule.add(c)
        
        # Gestion du pool
        d = pool_agent(4,self,self.n_pool,self.radius_pool)
        self.schedule.add(d)
        
        
        # Gestion des courbes
        # e2 = graphic_agent(6,self,b.solution, self.list_clients)
        # e3 = graphic_agent(7,self,c.solution, self.list_clients)
        #e = graphic_agent()
       # self.schedule.add(e1)
        # self.schedule.add(e2)
        # self.schedule.add(e3)
    def closing(self):
        for x in self.pool :
            x = closing_tour(x)
            
    def step(self):
        #passage de l'instant t à l'instant (t+1)
        self.schedule.step()
        self.closing()
        



In [ ]:
steps = 10

n_pool = 1
radius_pool = 20

model = SMA_collab(nb_pop, nb_generations, trucks_list, list_clients,c_matrix, t_matrix, n_pool,radius_pool)


for i in range(steps):
    model.step()